![Digital Earth Pacific](dep.png)

### Digital Earth Pacific Notebook

In [ ]:
import geopandas as gpd
import joblib
import numpy as np
import odc.geo.xr  # noqa
from dask import config
from dask.distributed import Client as dask_client
from odc.stac import load
from pystac_client import Client
from shapely import geometry
import depal_mh as dep
from depal_mh import predict_xr
from model import get_overlay
from ipyleaflet import Map, LayersControl, basemaps
import xarray as xr
import xrspatial.multispectral as ms
import warnings
import leafmap.leafmap as leafmap
import leafmap.colormaps as cm
from matplotlib import colors
warnings.filterwarnings('ignore')

Your code goes in the cells below. Add more cells here by clicking on the + button above.

In [ ]:
%reload_ext autoreload
%autoreload 2

config.set({'distributed.worker.daemon': False})

In [ ]:
aoi = dep.get_area_from_geojson("Marshall_Islands/laura_extent.geojson")
#aoi.explore()

In [ ]:
bbox = dep.get_bbox(aoi)
bbox_geometry = geometry.box(*bbox)
gdf = gpd.GeoDataFrame({'geometry': [bbox_geometry]}, crs='EPSG:4326')

In [ ]:
dep.list_data_bands("sentinel-2-l2a")

In [ ]:
catalog = "https://stac.staging.digitalearthpacific.org"
client = Client.open(catalog)
chunks = dict(x=2048, y=2048)

# Search for Sentinel-2 GeoMAD data
items = client.search(
    collections=["dep_s2_geomad"],
    bbox=bbox,
    datetime="2023",
).items()

# Load the data
data = load(items, chunks=chunks, bbox=bbox, resolution=10).squeeze("time")

#coastal clip
aoi = dep.get_country_admin_boundary("Marshall Islands", "Atoll", "Majuro")
data = dep.do_coastal_clip(aoi, data, buffer=0)
#data

In [ ]:

ndvi = ms.ndvi(data.B08, data.B04)
evi = ms.evi(data.B08, data.B04, data.B02)
arvi = ms.arvi(data.B08, data.B04, data.B02)
nbr2 = ms.nbr2(data.B11, data.B12)
savi = ms.savi(data.B08, data.B04)
ndmi = ms.ndmi(data.B08, data.B11)
ndwi = (data.B08 - data.B11) / (data.B08 - data.B11)

index = ndmi
index

In [ ]:
from datacube.utils.cog import write_cog
index.odc.write_cog("ndmi_laura_2023.tif", overwrite=True)

In [ ]:
index.plot.imshow(x="x", y="y", cmap="Blues_r")

In [ ]:
!pip install -q localtileserver

In [ ]:
m = leafmap.Map()
m.add_basemap("Esri.WorldImagery")
m.add_raster('ndmi_laura_2023.tif', colormap='Blues_r', bands=[1], layer_name='NDMI')
m.add_geojson('Marshall_Islands/laura_lens.geojson', layer_name="Groundwater Lenses")
m